In [1]:
%autosave 0

Autosave disabled


In [2]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

2022-11-26 23:09:57.599073: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-26 23:09:58.314703: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/clamytoe/miniconda3/envs/py39/lib/
2022-11-26 23:09:58.314793: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/clamytoe/miniconda3/envs/py39/lib/
2022-11-26 23:09:58.314802: W tensorflow/compile

In [3]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [4]:
from keras_image_helper import create_preprocessor

In [5]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [6]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [7]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [8]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [9]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [10]:
preds = pb_response.outputs['dense_7'].float_val

In [11]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [12]:
dict(zip(classes, preds))

{'dress': -1.8682903051376343,
 'hat': -4.761245250701904,
 'longsleeve': -2.316983461380005,
 'outwear': -1.0625708103179932,
 'pants': 9.887161254882812,
 'shirt': -2.8124334812164307,
 'shoes': -3.6662826538085938,
 'shorts': 3.200361728668213,
 'skirt': -2.6023378372192383,
 't-shirt': -4.835046291351318}